In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm import tqdm

In [2]:

from adn.data import load_datasets

train_ds, eval_ds = load_datasets(
    individuals_snp_dir = Path('data/output/script_test/individuals_snps'),
    metadata_path = Path("data/DIVRICE_3k10M_extended__project1__2025-02-12__6268161variants__HAPMAP/DIVRICE_3k10M_extended__3025individuals_metadata.tsv"),
    sequence_per_individual = 1000,
    sequence_length = 256,
    train_eval_split=0.2,
    data_ratio_to_use=0.1,
)

Loading SNP data...: 100%|██████████| 285/285 [00:01<00:00, 284.08it/s]
2025-02-26 00:31:19.791 | INFO     | adn.data:__init__:111 - Loaded 228 individuals
2025-02-26 00:31:19.793 | INFO     | adn.data:__init__:111 - Loaded 57 individuals


In [3]:
from adn.tokenizer import get_tokenizer



tokenizer = get_tokenizer()


In [8]:

from transformers import BertForSequenceClassification, BertConfig

dim = 128

config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    dim=dim,
    hidden_dim=4*dim,
    n_heads=4,
    num_labels=3,
)

model = BertForSequenceClassification(config)       

In [9]:
from torch.utils.data import DataLoader 

train_ds_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
eval_ds_loader = DataLoader(eval_ds, batch_size=8, shuffle=False)

In [10]:
import torch
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

def data_collator(features: list) -> dict:
    input_ids = torch.tensor([f[0] for f in features])
    attention_mask = torch.ones_like(input_ids)
    labels = torch.tensor([f[1] for f in features])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
)
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 